In [ ]:
#| default_exp framework_utils.fastai_cocolm
#| default_cls_lvl 3


In [ ]:
#| hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# framework_utils.fastai_cocolm

> As Jeremy says in his notebook, "Iteration speed is critical, so we need to quickly be able to try different data processing and trainer parameters" ... so lets create some helper methods to help us iterate more quickly.

This notebook is specific to MS coco-lm due to its lack of proper integration with HF transformers.

In [ ]:
#| export
import os, gc, time

from fastai.callback.all import *
from fastai.data.block import ColReader, ColSplitter, DataBlock, IndexSplitter, RegressionBlock
from fastai.imports import *
from fastai.layers import SigmoidRange
from fastai.learner import *
from fastai.losses import MSELossFlat
from fastai.optimizer import Adam
from fastai.metrics import *
from fastai.torch_core import *
from fastai.torch_imports import *
from transformers import AutoConfig, AutoModelForMaskedLM, AutoModelForSequenceClassification, logging

from blurr.callbacks import GradientCheckpointing
from blurr.text.data.core import TextBlock
from blurr.text.modeling.core import BaseModelWrapper, BaseModelCallback, blurr_splitter
from blurr.text.utils import get_hf_objects
from blurr.utils import PreCalculatedMSELoss, set_seed

from kaggle_comp import utils
from kaggle_comp.framework_utils import fastai
from kaggle_comp.cocolm.modeling_cocolm import COCOLMModel, COCOLMForSequenceClassification
from kaggle_comp.cocolm.configuration_cocolm import COCOLMConfig
from kaggle_comp.cocolm.tokenization_cocolm import COCOLMTokenizer


In [ ]:
#| hide_input
import pdb
from fastcore.test import *


In [ ]:
#| hide
#| cuda
torch.cuda.set_device(0)
print(f"Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}")


Using GPU #0: NVIDIA GeForce RTX 3090


## Hugging Face `transformers` objects

In [ ]:
#| export
def get_comp_hf_objects(
    model_checkpoint,
    model_cls=AutoModelForSequenceClassification,
    config_kwargs={"num_labels": 6},
    tokenizer_kwargs={},
    model_kwargs={},
    new_special_tokens=None,
):

    # need to create configuration object separately because we may be adding new attributes (e.g., cls_dropout)
    hf_arch = "cocolm"
    hf_config = COCOLMConfig.from_pretrained("microsoft/cocolm-base", **config_kwargs)
    hf_tokenizer = COCOLMTokenizer.from_pretrained("microsoft/cocolm-base", **tokenizer_kwargs)
    hf_model= COCOLMForSequenceClassification.from_pretrained("microsoft/cocolm-base", config=hf_config, **model_kwargs)

    if new_special_tokens:
        # After adding the new tokens, we need to resize the embedding matrix in the model and initialize the weights
        hf_tokenizer.add_special_tokens({"additional_special_tokens": new_special_tokens})
        hf_model.resize_token_embeddings(len(hf_tokenizer))

        with torch.no_grad():
            emb_size = hf_model.config.to_dict().get("embedding_size", hf_model.config.hidden_size )
            hf_model.get_input_embeddings().weight[-len(hf_tokenizer), :] = torch.zeros([emb_size])

    return hf_arch, hf_config, hf_tokenizer, hf_model


## Models

In [ ]:
#| export
class CocoLmBaseModelWrapper(Module):
    def __init__(
        self,
        # Your Hugging Face model
        hf_model,
        # If True, hidden_states will be returned and accessed from Learner
        output_hidden_states: bool = False,
        # If True, attentions will be returned and accessed from Learner
        output_attentions: bool = False,
        # Any additional keyword arguments you want passed into your models forward method
        hf_model_kwargs={},
    ):
        super().__init__()

        store_attr()
        self.hf_model = hf_model.cuda() if torch.cuda.is_available() else hf_model
        self.hf_model_fwd_args = list(inspect.signature(self.hf_model.forward).parameters.keys())

    def forward(self, x):
        for k in list(x):
            if k not in self.hf_model_fwd_args:
                del x[k]

        res = self.hf_model(
            **x,
             **self.hf_model_kwargs
        )

        if len(res) > 1:
            return {"loss": res[0], "logits": res[1]}
        else:
            return {"logits": res[0]}

## `Learner`

In [ ]:
#| export
def get_learner(
    dls,
    hf_model,
    custom_base_model=None,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_eps=1e-6,
    weight_decay=0.01,
    splitter=blurr_splitter,
    use_fp16=True,
    seed=utils.default_seed,
):
    set_seed(seed)

    model = CocoLmBaseModelWrapper(hf_model)
    loss_func = utils.MCRMSELoss()
    learn_cbs = [BaseModelCallback]

    learn = Learner(
        dls,
        model,
        opt_func=partial(Adam, mom=adam_beta1, sqr_mom=adam_beta2, eps=adam_eps, wd=weight_decay),
        moms=(adam_beta1, adam_beta1 - 0.10, adam_beta1),
        cbs=learn_cbs,
        splitter=splitter,
    )

    learn.loss_func = loss_func
    learn.metrics = [utils.MCRMSE()]
    learn.create_opt()
    learn.freeze()

    if use_fp16:
        learn = learn.to_fp16()

    return learn


In [ ]:
#| export
class FastaiCocoLmCompTrainer(utils.CompTrainer):
    def __init__(self, train_config, model_name, model_output_path="models", log_output_path="logs", **kwargs):
        super().__init__(
            train_config=train_config, model_name=model_name, model_output_path=model_output_path, log_output_path=log_output_path, **kwargs
        )

    def train(
        self,
        CFG,
        data,
        experiment_name: str,
        n_fold: int,
        run_id: str,
        grid_id: int,
        seed=None,
        verbose: bool = True,
    ):
        # timing
        start = time.time()

        if verbose:
            print(f"Experiment: {experiment_name}")
            print(f"Run: {run_id} | Grid ID: {grid_id} | Fold: {n_fold}")
            print(f"Training config: f{self.train_config}")

        if seed is None:
            seed = CFG.random_seed

        # from the experiment's training config if not in CFG
        # ------
        model_checkpoint = self.train_config.get("model_checkpoint", None)
        using_pretrained = self.train_config.get("using_pretrained", False)
        hf_config_kwargs = self.get_value_for("hf_config_kwargs", CFG, {"num_labels": 6})
        hf_tokenizer_kwargs = self.get_value_for("hf_tokenizer_kwargs", CFG, {})

       # datablock/dataloaders
        max_length = self.get_value_for("max_length", CFG, 256)
        include_labels = self.get_value_for("include_labels", CFG, True)
        tok_kwargs = self.get_value_for("tok_kwargs", CFG, {})
        batch_size = self.get_value_for("batch_size", CFG, 4)

        # learner
        adam_beta1 = self.get_value_for("adam_beta1", CFG, 0.9)
        adam_beta2 = self.get_value_for("adam_beta2", CFG, 0.999)
        adam_eps = self.get_value_for("adam_eps", CFG, 1e-6)
        weight_decay = self.get_value_for("weight_decay", CFG, 0.01)
        max_grad_norm = self.get_value_for("max_grad_norm", CFG, None)
        splitter = self.get_value_for("splitter", CFG, blurr_splitter)
        save_best_model = self.get_value_for("save_best_model", CFG, True)
        use_fp16 = self.get_value_for("use_fp16", CFG, True)
        custom_base_model_cls = self.get_value_for("custom_base_model_cls", CFG, None)
        custom_base_model_kwargs = self.get_value_for("custom_base_model_kwargs", CFG, {})

        # training
        n_frozen_epochs = self.get_value_for("n_frozen_epochs", CFG, 0)
        frozen_lr = self.get_value_for("frozen_lr", CFG, 1e-2)
        n_unfrozen_epochs = self.get_value_for("n_unfrozen_epochs", CFG, 5)
        unfrozen_lrs = self.get_value_for("unfrozen_lrs", CFG, [1e-5, 1e-2])
        # ------
        
        target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

        df = data.copy()
        
        # step 1: hf objects
        if verbose:
            print("Building HF objects ...")

        # WTG: Can't figure out how to add tokens to COCOLM
        new_special_tokens = None

        set_seed(seed)
        hf_arch, hf_config, hf_tokenizer, hf_model = get_comp_hf_objects(
            model_checkpoint, config_kwargs=hf_config_kwargs, tokenizer_kwargs=hf_tokenizer_kwargs, new_special_tokens=new_special_tokens
        )

        # step 2: dataloaders
        if verbose:
            print("Building DataLoaders ...")

        if not using_pretrained:
            get_x_func = partial(fastai.build_inputs)
        else:
            get_x_func = partial(fastai.build_inputs_like_pretrained)
            
        dls = fastai.get_dls(
            df,
            hf_arch,
            hf_config,
            hf_tokenizer,
            hf_model,
            get_x_func=get_x_func,
            val_idxs_or_fold=n_fold,
            max_length=max_length,
            include_labels=include_labels,
            tok_kwargs=tok_kwargs,
            batch_size=batch_size,
            seed=seed,
            augment=CFG.augment
        )

        # step 3: train
        if verbose:
            print("Building Learner ...")

        base_model_wrapper = None
        if custom_base_model_cls:
            base_model_wrapper = custom_base_model_cls(hf_config=hf_config, hf_model=hf_model, **custom_base_model_kwargs)

        self.learn = get_learner(
            dls,
            hf_model,
            custom_base_model=base_model_wrapper,
            adam_beta1=adam_beta1,
            adam_beta2=adam_beta2,
            adam_eps=adam_eps,
            weight_decay=weight_decay,
            splitter=splitter,
            use_fp16=use_fp16,
            seed=seed,
        )

        fit_cbs = []
        if max_grad_norm:
            fit_cbs.append(GradientClip(max_norm=max_grad_norm))

        if save_best_model:
            fit_cbs.append(
                SaveModelCallback(
                    monitor="mcrmse",
                    comp=np.less,
                    fname=f"{self.model_name}_best_mcrsme",
                    reset_on_fit=False,
                )
            )

        if n_frozen_epochs > 0:
            set_seed(seed)
            self.learn.fit_one_cycle(n_frozen_epochs, lr_max=frozen_lr, cbs=fit_cbs)

        if n_unfrozen_epochs > 0:
            self.learn.unfreeze()
            set_seed(seed)
            self.learn.fit_one_cycle(n_unfrozen_epochs, lr_max=slice(*unfrozen_lrs), cbs=fit_cbs)

        # step 4: log run details and performance on OOF data
        log_df = pd.DataFrame(self.learn.recorder.values, columns=self.learn.recorder.metric_names[1:-1])
        log_df["epoch"] = log_df.index
        log_df["fold"] = n_fold
        log_df["experiment"] = experiment_name
        log_df["run_id"] = run_id
        log_df["grid_id"] = grid_id
        log_df["subset"] = CFG.subset
        log_df["seed"] = seed
        log_df["batch_size"] = batch_size
        log_df["max_length"] = max_length
        log_df["n_frozen_epochs"] = n_frozen_epochs
        log_df["frozen_lr"] = frozen_lr
        log_df["n_unfrozen_epochs"] = n_unfrozen_epochs
        log_df["unfrozen_lrs"] = str(unfrozen_lrs)[1:-1]
        log_df["adam_beta1"] = adam_beta1
        log_df["adam_beta2"] = adam_beta2
        log_df["adam_beta2"] = adam_beta2
        log_df["adam_eps"] = adam_eps
        log_df["weight_decay"] = weight_decay
        log_df["max_grad_norm"] = max_grad_norm
        log_df["save_best_model"] = save_best_model
        log_df["use_fp16"] = use_fp16
        #log_df["strat_feat"] = CFG.strat_feat
        log_df["preprocess"] = CFG.preprocess
        log_df["postprocess"] = CFG.postprocess
        log_df["augment"] = CFG.augment

        # oof df
        preds, targs = self.learn.get_preds()
        preds = pd.DataFrame(preds)
        preds.columns = ["pred_" + x for x in target_cols]
        targs = pd.DataFrame(targs)
        targs.columns = ["targ_" + x for x in target_cols]
        oof_df = pd.concat([preds, targs], axis = 1)
        oof_df["run_id"] = run_id
        oof_df["grid_id"] = grid_id

        # time keeping
        log_df["time"] = time.time() - start

        # save model
        if verbose:
            print("--- saving model ---")

        self.learn.metrics = None
        self.learn.export(self.model_output_path / f"{self.model_name}.pkl")

        # clean up
        del self.learn, dls
        torch.cuda.empty_cache()
        gc.collect()

        return log_df, oof_df

    def predict(self, model_name, data):
        pass


## Export -

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()
